In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

In [2]:
# ======= Dataset =======
class CarlaDataset(Dataset):
    def __init__(self, images_path, angles_path, signals_path, transform=None):
        self.images = np.load(images_path)  # (N, 3, 224, 224)
        self.angles = np.load(angles_path).astype(np.float32)  # (N,)
        self.signals = np.load(signals_path).astype(np.float32)  # (N,)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]  # Already (3, 224, 224)
        angle = self.angles[idx]
        signal = self.signals[idx]

        if self.transform:
            img = self.transform(img)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(signal).unsqueeze(0), torch.tensor(angle)


In [ ]:
# ======= Model =======
class SteeringModel(nn.Module):
    def __init__(self):
        super(SteeringModel, self).__init__()
        # Load ResNet18 backbone without final FC
        resnet = models.resnet18(pretrained=True)
        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])  # Output: (B, 512, 1, 1)
        
        self.signal_fc = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU()
        )

        self.combined_fc = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, image, signal):
        x = self.cnn_backbone(image)  # (B, 512, 1, 1)
        x = x.view(x.size(0), -1)     # (B, 512)

        s = self.signal_fc(signal)   # (B, 32)

        combined = torch.cat([x, s], dim=1)  # (B, 544)
        out = self.combined_fc(combined)     # (B, 1)
        return out.squeeze(1)  # (B,)

In [ ]:
# ======= Transforms =======
transform = transforms.Compose([
    transforms.ToTensor(),  # if images aren't tensors already
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ResNet ImageNet means
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# ======= Dataloader =======
dataset = CarlaDataset('images.npy', 'angles.npy', 'turn_signals.npy')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# ======= Training Setup =======
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SteeringModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [ ]:
# ======= Training Loop =======
for epoch in range(10):
    model.train()
    total_loss = 0
    for images, signals, angles in dataloader:
        images, signals, angles = images.to(device), signals.to(device), angles.to(device)

        optimizer.zero_grad()
        preds = model(images, signals)
        loss = criterion(preds, angles)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")